<a href="https://colab.research.google.com/github/hiroalchem/data_science_lecture/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データサイエンス講義第三回
これはデータサイエンス講義第二回用スクリプトです   
Colaboratoryのハードウェア アクセラレータ設定は今回はCPU (None) にして実行してください   
上のランタイムタブからランタイムのタイプを変更を選択してください   
Google Drive をマウントしてください

このスクリプトでは学習結果の評価を行ってみます

In [13]:
# lectureのディレクトリに移動します
%cd /content/drive/MyDrive/lecture2020

/content/drive/MyDrive/lecture2020


In [2]:
import os
import pathlib

In [3]:
# 必要なライブラリをインストールします
!pip install pycocotools
!pip install tf_slim
!pip install lvis
!pip install tensorflow==2.2

     |████████████████████████████████| 358kB 13.2MB/s 
     |████████████████████████████████| 516.2MB 20kB/s 
     |████████████████████████████████| 3.0MB 49.2MB/s 
     |████████████████████████████████| 460kB 54.4MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [4]:
!rm -r -f /content/drive/MyDrive/lecture2020/models
# tensorflow のリポジトリをクローンしていなければ https://github.com/tensorflow/models をクローンします

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2322, done.
remote: Counting objects: 100% (2322/2322), done.
remote: Compressing objects: 100% (2026/2026), done.
remote: Total 2322 (delta 562), reused 921 (delta 271), pack-reused 0
Receiving objects: 100% (2322/2322), 30.65 MiB | 2.13 MiB/s, done.
Resolving deltas: 100% (562/562), done.


In [7]:
# 必要なものをインストールします
# 少し時間がかかります
%%bash
cd /content/drive/MyDrive/lecture2020/models/research
protoc object_detection/protos/*.proto --python_out=.
pip install .

Processing /content/drive/MyDrive/lecture2020/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1376510 sha256=417e390eecc7b019ea4c6241325dc25cf6a6c6997b50ad8004eda8937a6c9656
  Stored in directory: /tmp/pip-ephem-wheel-cache-oxp8kzvz/wheels/d3/e5/d8/a0ccb834fbf73ce1aac5cd232242e48dbdd8a11015e7c6d136
Successfully built object-detection


In [8]:
# install できたか確認
%cd /content/drive/MyDrive/lecture2020/models/research
%set_env PYTHONPATH=/content/drive/MyDrive/lecture2020/models/research:/content/drive/MyDrive/lecture2020/models/research/slim
os.environ['PYTHONPATH'] += ":/content/drive/MyDrive/lecture2020/models"
import sys
sys.path.append("/content/drive/MyDrive/lecture2020/models")
!python ./object_detection/builders/model_builder_tf2_test.py
%cd /content/drive/MyDrive/lecture2020

/content/drive/MyDrive/lecture2020/models/research
env: PYTHONPATH=/content/drive/MyDrive/lecture2020/models/research:/content/drive/MyDrive/lecture2020/models/research/slim
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2020-12-08 02:55:46.784812: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-08 02:55:46.848882: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-08 02:55:46.848951: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (43703623af7a): /proc/driver/nvidia/version does not exist
2020-12-08 02:55:46.849327: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-12-08 02:55:46.85662

In [12]:
%cd /content/drive/MyDrive/lecture2020/models/research
!python object_detection/model_main_tf2.py \
    --pipeline_config_path="/content/drive/MyDrive/lecture2020/trial1/mypipeline.config" \
    --model_dir="/content/drive/MyDrive/lecture2020/trial1/models/faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8" \
    --checkpoint_dir="/content/drive/MyDrive/lecture2020/trial1/models/faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8" \
    --alsologtostderr

/content/drive/MyDrive/lecture2020/models/research
W1208 03:13:11.249118 140556562605952 model_lib_v2.py:1026] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I1208 03:13:11.249445 140556562605952 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1208 03:13:11.249566 140556562605952 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1208 03:13:11.249686 140556562605952 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W1208 03:13:11.249841 140556562605952 model_lib_v2.py:1041] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
2020-12-08 03:13:12.054712: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-12-08 

Avereage PrecisionやAverage Recallが出てきたと思います   
出てきたら上のセルは止めてください

この中で一番よく使われるのは Average Precision (AP)です   
APとは簡単には推論したboxがどのくらい正解だったかの指標です   
詳細はこちらなどを読んで見てください
https://qiita.com/mdo4nt6n/items/08e11426e2fac8433fed  
これを求めるには推論したboxがどの程度正解のboxと重なっていた場合に正解とするかの閾値を決める必要があります   
それが IoU (Intersection over Union) 
一番わかりやすいのは2つ目のIoU=0.50と書いてある項目で、つまり半分以上重なっていたら正解とする時のAPを求めた時の数値、のようなものです


学習step数を増やした場合や、別のデータセットで見た目の結果とAPの数値を比較してみてください